In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

In [2]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_n.csv'
df_n = pd.read_csv(path)
df_n.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'experience', 'supplies', 'insured', 'involved',
       'learned_routes', 'made_plan', 'made_safer', 'planned_neighbors',
       'practiced_drills', 'documents', 'rainy_day', 'alerts',
       'family_communication', 'none', 'dont_know', 'age', 'sex', 'education',
       'race', 'homeownership', 'income', 'rentmortgage', 'rurality',
       'hazard_weight', 'geometry', 'zip_count'],
      dtype='object')

In [3]:
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_k.csv'
df_k = pd.read_csv(path)
df_k.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'awareness', 'perception', 'experience', 'floodzone',
       'efficacy', 'supplies', 'insured', 'involved', 'learned_routes',
       'made_plan', 'made_safer', 'planned_neighbors', 'practiced_drills',
       'documents', 'rainy_day', 'alerts', 'family_communication', 'none',
       'dont_know', 'age', 'sex', 'education', 'race', 'homeownership',
       'income', 'rentmortgage', 'rurality', 'hazard_weight', 'geometry',
       'zip_count'],
      dtype='object')

#### We will go through these outcome variables one by one

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- learned_routes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication

In [4]:
def probit(functions, data):
    results_list = []
    for func in functions:
        model = smf.probit(formula=func, data=data).fit(disp=0)

        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,                   
            'pseudoR_2': model.prsquared,
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results


In [5]:
#duplicate but with logit
def logit(functions, data):
    results_list = []
    for func in functions:
        model = smf.logit(formula=func, data=data).fit(disp=0)
        marg_effects = model.get_margeff().summary_frame()

        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,                   
            'marginal_effect': marg_effects['dy/dx'],
            'pseudoR_2': model.prsquared,
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results

In [6]:

functions = [
    'made_safer ~ age + income + education + homeownership',
    'insured ~ age + income + education + homeownership',
    'learned_routes ~ age + income + education + homeownership',
    'supplies ~ age + income + education + homeownership',
    'involved ~ age + income + education + homeownership',
    'made_plan ~ age + income + education + homeownership',
    'practiced_drills ~ age + income + education + homeownership',
    'alerts ~ age + income + education + homeownership',
    'family_communication ~ age + income + education + homeownership'    
]

In [7]:
results = probit(functions=functions, data=df_n)
results.to_excel('probit_socioeconomic.xlsx')

In [8]:
results = logit(functions=functions, data=df_n)
results.to_excel('logit_socioeconomic.xlsx')